In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
!pip list

In [ ]:
!pip install opencv-python

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
from ultralytics import YOLO

In [ ]:
#Load Model
#model=torch.hub.load('yolov8n.pt')
model=YOLO('yolov8n.pt')

In [ ]:
model.fuse()

In [ ]:
model.predict(
   source='https://media.roboflow.com/notebooks/examples/dog.jpeg',
   conf=0.25
)

In [ ]:
img='https://media.roboflow.com/notebooks/examples/dog.jpeg'

In [ ]:
results = model(img)
print(results)

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model.predict(frame)
    
    cv2.imshow('YOLO',results)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
!pip install supervisor

In [ ]:
!pip install supervision==0.1.0

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

class ObjectDetection:
    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device:",self.device)
              
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names
        self.box_annotator = BoxAnnotator(color=ColorPlette(), thickness=3, text_thickness=3, text_scale=1.5)
              
    def load_modal(self):
              model=YOLO("yolo8n.pt")
              model.fuse()
              
              return model
    
    def predict(self, frame):
              results=self.model(frame)
              return results
              
    def plot_bboxes(self, results, frame):
              xyxys=[]
              confidences=[]
              class_ids=[]
              
              #Extract detection for person class
              for result in results[0]:
                  class_id=result.boxes.cls.cpu().numpy().astype(int)
                  
                  if class_id==0:
                      xyxys.append(result.boxes.xyxy.cpu().numpy())
                      confidences.append(result.boxes.conf.cpu().numpy())
                      class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
              
              detections=Detections(xyxy=results[0].boxes.xyxy.cpu().numpy(),
                                   confidence=results[0].boxes.conf.cpu().numpy(),
                                   class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                                   )
              
              #Format custom labels
              self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"for _, confidence, class_id, tracker_id in detections]
              
              #Annotate and display frame
              frame=self.box_annotator.annotate(frame=frame, detections=detections, labels=self.labels)
              return frame
              
    def __call__(self):
            cap = cv2.VideoCapture(self.capture_index)
            assert icap.isOpened()
            cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            
            while True:
              start_time = time()
              
              ret, frame = cap.read()
              assert ret
              
              results = self.predict(frame)
              frame = self.plot_bboxes (results, frame)
              
              end_time = time()
              fps=1/np.round(end_time - start_time, 2)
              
              cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5)
              
              cv2.imshow("YOLOv8 Detection",frame)
              
              if cv2.waitKey (5) & 0xFF == 27:
                      break
            
            cap.release()
            cv2.destroyAllWindows()

detector=ObjectDetection(capture_index=0)
detector()

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

class ObjectDetection:
    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device:",self.device)
              
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names
        self.box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)
              
    def load_model(self):
        #model=YOLO("yolov8n.pt")
        model=YOLO("yolov8l.pt")
        #model.fuse()
        
        return model
    
    def predict(self, frame):
        results=self.model(frame)
        return results
              
    def plot_bboxes(self, results, frame):
        xyxys=[]
        confidences=[]
        class_ids=[]
        
        #Extract detection for person class
        for result in results[0]:
            class_id=result.boxes.cls.cpu().numpy().astype(int)
            
            if class_id==0:
                xyxys.append(result.boxes.xyxy.cpu().numpy())
                confidences.append(result.boxes.conf.cpu().numpy())
                class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
        
        detections=Detections(xyxy=results[0].boxes.xyxy.cpu().numpy(),
                            confidence=results[0].boxes.conf.cpu().numpy(),
                            class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                            )
        
        #Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"for _, confidence, class_id, tracker_id in detections]
        
        #Annotate and display frame
        frame=self.box_annotator.annotate(frame=frame, detections=detections, labels=self.labels)
        return frame
              
    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        while True:
            start_time = time()
            
            ret, frame = cap.read()
            assert ret
            
            results = self.predict(frame)
            frame = self.plot_bboxes (results, frame)
            
            end_time = time()
            fps=1/np.round(end_time - start_time, 2)
            
            #cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5)
            
            cv2.imshow("YOLOv8 Detection",frame)
            
            if cv2.waitKey (5) & 0xFF == 27:
                    break
        
        cap.release()
        cv2.destroyAllWindows()

detector=ObjectDetection(capture_index=0)
detector()

## Working Condn

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

def load_model():
    # model = YOLO("yolov8n.pt")
    model = YOLO("yolov8l.pt")
    # model.fuse()
    return model

def predict(model, frame):
    results = model(frame)
    return results

def plot_bboxes(results, frame, class_names_dict, box_annotator):
    xyxys = []
    confidences = []
    class_ids = []

    # Extract detection for person class
    for result in results[0]:
        class_id = result.boxes.cls.cpu().numpy().astype(int)

        if class_id == 0:
            xyxys.append(result.boxes.xyxy.cpu().numpy())
            confidences.append(result.boxes.conf.cpu().numpy())
            class_ids.append(result.boxes.cls.cpu().numpy().astype(int))

    detections = Detections(xyxy=results[0].boxes.xyxy.cpu().numpy(),
                            confidence=results[0].boxes.conf.cpu().numpy(),
                            class_id=results[0].boxes.cls.cpu().numpy().astype(int))

    # Format custom labels
    labels = [f"{class_names_dict[class_id]} {confidence:0.2f}" for _, confidence, class_id, tracker_id in detections]

    # Annotate and display frame
    frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
    return frame

def run_object_detection(capture_index):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using Device:", device)

    model = load_model()
    class_names_dict = model.model.names
    box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)

    cap = cv2.VideoCapture(capture_index)
    assert cap.isOpened()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while True:
        start_time = time()

        ret, frame = cap.read()
        assert ret

        results = predict(model, frame)
        frame = plot_bboxes(results, frame, class_names_dict, box_annotator)

        end_time = time()
        fps = 1 / np.round(end_time - start_time, 2)

        # cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5)

        cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(5) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

run_object_detection(capture_index=0)


In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

def load_model():
    # model = YOLO("yolov8n.pt")
    model = YOLO("yolov8l.pt")
    # model.fuse()
    return model

def predict(model, frame):
    results = model(frame)
    return results

def plot_bboxes(results, frame, class_names_dict, box_annotator, threshold):
    xyxys = []
    confidences = []
    class_ids = []

    # Extract detection for person class
    for result in results.xyxy[0]:
        class_id = int(result[5])
        confidence = float(result[4])

        if class_id == 0 and confidence >= threshold:
            xyxys.append(result[:4])
            confidences.append(confidence)
            class_ids.append(class_id)

    detections = Detections(xyxy=np.array(xyxys),
                            confidence=np.array(confidences),
                            class_id=np.array(class_ids))

    # Format custom labels
    labels = [f"{class_names_dict[class_id]} {confidence:0.2f}" for _, confidence, class_id in zip(detections.xyxy, detections.confidence, detections.class_id)]

    # Annotate and display frame
    frame = box_annotator.annotate(frame=frame, detections=detections.xyxy, labels=labels)
    return frame

def run_object_detection(capture_index, threshold):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using Device:", device)

    model = load_model()
    class_names_dict = model.model.names
    box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)

    cap = cv2.VideoCapture(capture_index)
    assert cap.isOpened()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while True:
        start_time = time()

        ret, frame = cap.read()
        assert ret

        results = predict(model, frame)
        frame = plot_bboxes(results, frame, class_names_dict, box_annotator, threshold)

        end_time = time()
        fps = 1 / np.round(end_time - start_time, 2)

        # cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5)

        cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(5) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

run_object_detection(capture_index=0, threshold=0.5)


# Socket Programing

## Server

In [ ]:
!pip install imutils

In [ ]:
!pip install pyshine

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

def load_model():
    # model = YOLO("yolov8n.pt")
    model = YOLO("yolov8l.pt")
    # model.fuse()
    return model

def predict(model, frame):
    results = model(frame)
    return results

def plot_bboxes(results, frame, class_names_dict, box_annotator):
    xyxys = []
    confidences = []
    class_ids = []

    # Extract detection for person class
    for result in results[0]:
        class_id = result.boxes.cls.cpu().numpy().astype(int)

        if class_id == 0:
            xyxys.append(result.boxes.xyxy.cpu().numpy())
            confidences.append(result.boxes.conf.cpu().numpy())
            class_ids.append(result.boxes.cls.cpu().numpy().astype(int))

    detections = Detections(xyxy=results[0].boxes.xyxy.cpu().numpy(),
                            confidence=results[0].boxes.conf.cpu().numpy(),
                            class_id=results[0].boxes.cls.cpu().numpy().astype(int))

    # Format custom labels
    labels = [f"{class_names_dict[class_id]} {confidence:0.2f}" for _, confidence, class_id, tracker_id in detections]

    # Annotate and display frame
    frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
    return frame

def run_object_detection(frame,capture_index):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using Device:", device)

    model = load_model()
    class_names_dict = model.model.names
    box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)

    cap = cv2.VideoCapture(capture_index)
    assert cap.isOpened()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while True:
        start_time = time()

        ret, frame = cap.read()
        assert ret

        results = predict(model, frame)
        frame = plot_bboxes(results, frame, class_names_dict, box_annotator)

        end_time = time()
        fps = 1 / np.round(end_time - start_time, 2)

        # cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5)

        cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(5) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

#run_object_detection(capture_index=0)


In [1]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

In [2]:
def load_model():
    # model = YOLO("yolov8n.pt")
    model = YOLO("yolov8l.pt")
    # model.fuse()
    return model

In [3]:
def predict(model, frame):
    results = model(frame)
    return results

In [4]:
def plot_bboxes(results, frame, class_names_dict, box_annotator):
    xyxys = []
    confidences = []
    class_ids = []

    # Extract detection for person class
    for result in results[0]:
        class_id = result.boxes.cls.cpu().numpy().astype(int)

        if class_id == 0:
            xyxys.append(result.boxes.xyxy.cpu().numpy())
            confidences.append(result.boxes.conf.cpu().numpy())
            class_ids.append(result.boxes.cls.cpu().numpy().astype(int))

    detections = Detections(xyxy=results[0].boxes.xyxy.cpu().numpy(),
                            confidence=results[0].boxes.conf.cpu().numpy(),
                            class_id=results[0].boxes.cls.cpu().numpy().astype(int))

    # Format custom labels
    labels = [f"{class_names_dict[class_id]} {confidence:0.2f}" for _, confidence, class_id, tracker_id in detections]

    # Annotate and display frame
    frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
    return frame

In [5]:
def run_object_detection(frame,capture_index):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using Device:", device)

    model = load_model()
    class_names_dict = model.model.names
    box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)

    cap = cv2.VideoCapture(capture_index)
    assert cap.isOpened()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while True:
        start_time = time()

        ret, frame = cap.read()
        assert ret

        results = predict(model, frame)
        frame = plot_bboxes(results, frame, class_names_dict, box_annotator)

        end_time = time()
        fps = 1 / np.round(end_time - start_time, 2)

        # cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5)

        cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(5) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Here the server is receiving video from the client
import socket
import cv2
import pickle
import struct
import pyshine as ps
import imutils
import threading

server_socket=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host=socket.gethostname()
ip_add=socket.gethostbyname(host)

print("Host IP: ",ip_add)
port=1234

socket_address=(ip_add, port)
server_socket.bind(socket_address)
server_socket.listen()
print("Listening at ",socket_address)

def show_client(addr, client_socket):
    try:
        print("Client {} Connected".format(addr))
        if client_socket:  #if client socket exists
            data=b""
            payload_size=struct.calcsize("Q")

            while True:
                while len(data)<payload_size:
                    packet=client_socket.recv(4*1024) #4K
                    if not packet:
                        break
                    data+=packet
                packed_msg_size=data[:payload_size]
                data=data[payload_size:]
                msg_size=struct.unpack("Q", packed_msg_size)[0]

                while len(data)<msg_size:
                    data+=client_socket.recv(4*1024)
                frame_data=data[:msg_size]
                data=data[msg_size:]
                frame=pickle.loads(frame_data)
                
                #model = load_model()
                #class_names_dict = model.model.names
                #box_annotator = BoxAnnotator(color=ColorPalette(), thickness=3, text_thickness=3, text_scale=1.5)

                #cap = cv2.VideoCapture(capture_index)
                #assert cap.isOpened()
                #cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
                #cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
                
                #results = predict(model, frame)
                #frame = plot_bboxes(results, frame, class_names_dict, box_annotator)
                cv2.imshow(frame)
                #text="Client: {}".format(addr)
                #frame=ps.putBText(frame, text, 10, 10, vspace=10, hspace=1, font_scale=0.7, background_RGB=(255,0,0), text_RGB=(255,250,250))
                #cv2.imshow("From {} {}".format(addr,frame))

                key=cv2.waitKey(1)& 0xFF
                if key==ord("q"):
                       client_socket.close()
    except Exception as e:
        print("Client {} Disconnected".format(addr))

while True:
    client_socket,addr=server_socket.accept()
    thread=threading.Thread(target=show_client, args=(addr, client_socket))
    thread.start()
    print("Total clients ",threading.activeCount()-1)